In [2]:
!pip install -r requirements.txt

In [64]:
import json
from bson import ObjectId
from datetime import datetime
from db_connection import get_db_connection

db = get_db_connection()

### Insert nového výjezdu
if db.addresses.find_one({"departure_id": 99999999}):
    print('Tato adresa již existuje')
else:
    db.addresses.insert_one({
    "departure_id": 99999999,
    "region": {"id": 116, "name": "Jihomoravský"},
    "district": {"id": 3706, "name": "Hodonín"},
    "municipality": "Dubňany",
    "municipalityPart": "Dubňany",
    "municipalityWithExtendedCompetence": "Dubňany",
    "street": "Hornická 1181",
    "gis1": "123456",
    "gis2": "654321",
    "preplanned": False,
    "road": ""
    })


if db.departures.find_one({"departure_id": 99999999}):
    print('Tento výjezd již existuje')
else:
    db.departures.insert_one({
    "departure_id": 99999999,
    "reportedDateTime": datetime.utcnow(),
    "state": "Likvidovaná",
    "type": "Záchrana osob a zvířat",
    "subType": "Záchrana, odchyt zvířat",
    "description": "Zneškodnění splašené kapybary",
    "region_url": "https://udalosti.firebrno.cz"
    })

db.sent_units.insert_many([
    {
        "departure_id": 99999999,
        "type": "pila řetězová",
        "unit": "HZS Dubňany",
        "count": 2,
        "currentCount": 2,
        "callDateTime": datetime.utcnow()
    },
    {
        "departure_id": 99999999,
        "type": "rychlý zásahový automobil",
        "unit": "HZS Dubňany",
        "count": 1,
        "currentCount": None,
        "callDateTime": datetime.utcnow()
    }
])

addresss = db.addresses.find_one({"departure_id": 99999999})
departure = db.departures.find_one({"departure_id": 99999999})
sent_units = db.sent_units.find({"departure_id": 99999999})

print('Adresa:')
print(json.dumps(addresss, default=str, indent=4, ensure_ascii=False))
print('Departure:')
print(json.dumps(departure, default=str, indent=4, ensure_ascii=False))
print('Vyslané jednotky:')
print(json.dumps(list(sent_units), default=str, indent=4, ensure_ascii=False))

Adresa:
{
    "_id": "682a3e3b4bcec12c61708d8f",
    "departure_id": 99999999,
    "region": {
        "id": 116,
        "name": "Jihomoravský"
    },
    "district": {
        "id": 3706,
        "name": "Hodonín"
    },
    "municipality": "Dubňany",
    "municipalityPart": "Dubňany",
    "municipalityWithExtendedCompetence": "Dubňany",
    "street": "Hornická 1181",
    "gis1": "123456",
    "gis2": "654321",
    "preplanned": false,
    "road": ""
}
Departure:
{
    "_id": "682a3e3b4bcec12c61708d90",
    "departure_id": 99999999,
    "reportedDateTime": "2025-05-18 20:08:27.985000",
    "state": "Likvidovaná",
    "type": "Záchrana osob a zvířat",
    "subType": "Záchrana, odchyt zvířat",
    "description": "Zneškodnění splašené kapybary",
    "region_url": "https://udalosti.firebrno.cz"
}
Vyslané jednotky:
[
    {
        "_id": "682a3e3b4bcec12c61708d91",
        "departure_id": 99999999,
        "type": "pila řetězová",
        "unit": "HZS Dubňany",
        "count": 2,
       

In [57]:
from db_connection import get_db_connection
db = get_db_connection()

### Update one
db.departures.update_one(
    {"departure_id": 99999999},
    {"$set": {"state": "Likvidovaná"}}
)


UpdateResult({'nModified': 0, 'n': 1, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1747598460, 1), 'signature': {'hash': b'\x02\xff_\t1o\xd5D@\xe9\x8a\xb3\x87\xb9\xec/=3T\xda', 'keyId': 7505753227216814103}}, 'operationTime': Timestamp(1747598452, 1), 'updatedExisting': True}, acknowledged=True)

In [58]:
from datetime import datetime, timedelta
from db_connection import get_db_connection
db = get_db_connection()

### Update many
current_datetime = datetime.utcnow()
# Odečtení 30 dní
date_30_days_ago = current_datetime - timedelta(days=30)
date_30_days_ago_iso = date_30_days_ago.isoformat() + 'Z'

db.departures.update_many(
    {
        "state": "Otevřená, SaP na místě",
        "reportedDateTime": {"$lt": date_30_days_ago_iso}
    },
    {"$set": {"state": "Ukončená - manuálně"}}
)


UpdateResult({'nModified': 0, 'n': 0, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1747598470, 1), 'signature': {'hash': b'\xe0\x16\xf6R-R\xcf\x90\xd0\xcb\xb5\x80E\xda\x9e\xebn\xd2\xa5;', 'keyId': 7505753227216814103}}, 'operationTime': Timestamp(1747598466, 1), 'updatedExisting': False}, acknowledged=True)

In [61]:
from db_connection import get_db_connection
db = get_db_connection()

### Delete many
db.addresses.delete_one({"departure_id": 99999999})
db.departures.delete_one({"departure_id": 99999999})
db.sent_units.delete_many({"departure_id": 99999999})

DeleteResult({'n': 58, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1747598721, 6), 'signature': {'hash': b'\xa8\xda\xb9\xc8\xf2\xdf\xf9\xb1\xf8\nRE\xb4\xef\x05=\x04l\xb6\x97', 'keyId': 7505753227216814103}}, 'operationTime': Timestamp(1747598721, 6)}, acknowledged=True)

In [70]:
from bson.json_util import dumps
from db_connection import get_db_connection
db = get_db_connection()

### Merge adresy a vyslaných jednotek
pipeline = [
    {"$match": {"departure_id": 99999999}},
    {"$lookup": {
        "from": "addresses",
        "localField": "departure_id",
        "foreignField": "departure_id",
        "as": "address"
    }},
    {"$unwind": {"path": "$address", "preserveNullAndEmptyArrays": True}},  # Rozbalení adresy
    {"$lookup": {
        "from": "sent_units",
        "localField": "departure_id",
        "foreignField": "departure_id",
        "as": "sent_units"
    }},
    {"$project": {
        "_id": 0,
        "departure_id": 1,
        "address": 1,
        "sent_units": 1
    }},
    {"$merge": {
        "into": "merged_address_units",
        "on": "departure_id",
        "whenMatched": "replace",
        "whenNotMatched": "insert"
    }}
]

db.merged_address_units.create_index("departure_id", unique=True)

# Spuštění agregace
db.departures.aggregate(pipeline)

merged = db.merged_address_units.find_one({"departure_id": 99999999})
print(json.dumps(merged, default=str, indent=4, ensure_ascii=False))

{
    "_id": "682a4153cff2ba40ee90162e",
    "departure_id": 99999999,
    "address": {
        "_id": "682a3e3b4bcec12c61708d8f",
        "departure_id": 99999999,
        "region": {
            "id": 116,
            "name": "Jihomoravský"
        },
        "district": {
            "id": 3706,
            "name": "Hodonín"
        },
        "municipality": "Dubňany",
        "municipalityPart": "Dubňany",
        "municipalityWithExtendedCompetence": "Dubňany",
        "street": "Hornická 1181",
        "gis1": "123456",
        "gis2": "654321",
        "preplanned": false,
        "road": ""
    },
    "sent_units": [
        {
            "_id": "682a3e3b4bcec12c61708d91",
            "departure_id": 99999999,
            "type": "pila řetězová",
            "unit": "HZS Dubňany",
            "count": 2,
            "currentCount": 2,
            "callDateTime": "2025-05-18 20:08:27.988000"
        },
        {
            "_id": "682a3e3b4bcec12c61708d92",
            "depart

In [67]:
from bson.json_util import dumps
from db_connection import get_db_connection
db = get_db_connection()

### Agregace sloučení záznamů do jedné kolekce
pipeline = [
    {"$match": {"departure_id": 99999999}},
    {"$lookup": {
        "from": "addresses",
        "localField": "departure_id",
        "foreignField": "departure_id",
        "as": "adresa"
    }},
    {"$lookup": {
        "from": "sent_units",
        "localField": "departure_id",
        "foreignField": "departure_id",
        "as": "jednotky"
    }},
    {"$project": {
        "_id": 0,
        "departure_id": 1,
        "reportedDateTime": 1,
        "state": 1,
        "type": 1,
        "subType": 1,
        "description": 1,
        "region_url": 1,
        "adresa": {"$arrayElemAt": ["$adresa", 0]},
        "jednotky": 1
    }}
]

result = list(db.departures.aggregate(pipeline))

if result:
    print("Nalezený záznam:")
    print(dumps(result[0], indent=2, ensure_ascii=False))
else:
    print("Žádné záznamy nenalezeny")

Nalezený záznam:
{
  "departure_id": 99999999,
  "reportedDateTime": {
    "$date": "2025-05-18T20:08:27.985Z"
  },
  "state": "Likvidovaná",
  "type": "Záchrana osob a zvířat",
  "subType": "Záchrana, odchyt zvířat",
  "description": "Zneškodnění splašené kapybary",
  "region_url": "https://udalosti.firebrno.cz",
  "jednotky": [
    {
      "_id": {
        "$oid": "682a3e3b4bcec12c61708d91"
      },
      "departure_id": 99999999,
      "type": "pila řetězová",
      "unit": "HZS Dubňany",
      "count": 2,
      "currentCount": 2,
      "callDateTime": {
        "$date": "2025-05-18T20:08:27.988Z"
      }
    },
    {
      "_id": {
        "$oid": "682a3e3b4bcec12c61708d92"
      },
      "departure_id": 99999999,
      "type": "rychlý zásahový automobil",
      "unit": "HZS Dubňany",
      "count": 1,
      "currentCount": null,
      "callDateTime": {
        "$date": "2025-05-18T20:08:27.988Z"
      }
    }
  ],
  "adresa": {
    "_id": {
      "$oid": "682a3e3b4bcec12c61708d8f"

In [77]:
from bson.json_util import dumps
from db_connection import get_db_connection
db = get_db_connection()

### Výjezdy s více než 50 vyslanými jednotkami, seřazené podle data
pipeline = [
    {"$lookup": {
        "from": "sent_units",
        "localField": "departure_id",
        "foreignField": "departure_id",
        "as": "units"
    }},
    # Přidáme počet jednotek
    {"$addFields": {
        "unitsCount": {"$size": "$units"}
    }},
    # Filtrujeme výjezdy s více než 50 jednotkami
    {"$match": {"unitsCount": {"$gt": 50}}},
    # Seřadíme podle data výjezdu
    {"$sort": {"reportedDateTime": 1}},
    # Vybereme relevantní pole
    {"$project": {
        "_id": 0,
        "departure_id": 1,
        "reportedDateTime": 1,
        "type": 1,
        "unitsCount": 1
    }}
]

result = list(db.departures.aggregate(pipeline))

if result:
    for res in result:
        print(dumps(res, indent=2, ensure_ascii=False))
else:
    print("Žádné záznamy nenalezeny")

{
  "departure_id": 344375062,
  "reportedDateTime": {
    "$date": "2021-03-20T23:08:42Z"
  },
  "type": "Požár",
  "unitsCount": 106
}
{
  "departure_id": 348826062,
  "reportedDateTime": {
    "$date": "2021-05-11T12:52:20Z"
  },
  "type": "Požár",
  "unitsCount": 69
}
{
  "departure_id": 351458062,
  "reportedDateTime": {
    "$date": "2021-06-09T10:01:18Z"
  },
  "type": "Požár",
  "unitsCount": 57
}
{
  "departure_id": 351660062,
  "reportedDateTime": {
    "$date": "2021-06-11T10:06:55Z"
  },
  "type": "Technická pomoc",
  "unitsCount": 90
}
{
  "departure_id": 363700062,
  "reportedDateTime": {
    "$date": "2021-10-05T19:11:16Z"
  },
  "type": "Požár",
  "unitsCount": 112
}
{
  "departure_id": 409391062,
  "reportedDateTime": {
    "$date": "2023-06-09T16:49:16Z"
  },
  "type": "Požár",
  "unitsCount": 100
}
{
  "departure_id": 422081062,
  "reportedDateTime": {
    "$date": "2023-11-01T13:15:30Z"
  },
  "type": "Požár",
  "unitsCount": 58
}
{
  "departure_id": 447977062,
  "r

In [108]:
from bson.json_util import dumps
from db_connection import get_db_connection
db = get_db_connection()

### Výjezdy, kde popis obsahuje slovo "hmyz" a počet vyslaných jednotek mezi určenými datumy
pipeline = [
    {"$match": {
        "description": {"$regex": "hmyz", "$options": "i"},
        "reportedDateTime": {
            "$gte": datetime(2008, 1, 1),
            "$lt": datetime(2015, 12, 31)
        }
    }
    },  # filtrovat podle "hmyz" (case-insensitive)
    {"$lookup": {
        "from": "addresses",
        "localField": "departure_id",
        "foreignField": "departure_id",
        "as": "address"
    }},
    {"$unwind": "$address"},  # rozbalení adresy
    {"$group": {
        "_id": "$address.district.name",  # seskupení podle názvu okresu
        "count": {"$sum": 1}               # spočítání výjezdů
    }},
    {"$project": {
        "district": "$_id",
        "count": 1,
        "_id": 0
    }},
    {"$sort": {"count": -1}}  # seřazení podle počtu sestupně
]

result = list(db.departures.aggregate(pipeline))

if result:
    for res in result:
        print(dumps(res, indent=2, ensure_ascii=False))
else:
    print("Žádné záznamy nenalezeny")

{
  "count": 205,
  "district": "Brno-město"
}
{
  "count": 201,
  "district": "Blansko"
}
{
  "count": 187,
  "district": "Brno-venkov"
}
{
  "count": 73,
  "district": "Znojmo"
}
{
  "count": 70,
  "district": "Vyškov"
}
{
  "count": 53,
  "district": "Hodonín"
}
{
  "count": 37,
  "district": "Břeclav"
}


In [112]:
from bson.json_util import dumps
from db_connection import get_db_connection
db = get_db_connection()

### Dny s největším počtem výjezdů za roky 2010 - 2015
pipeline = [
    # Filtrování na roky 2010-2015
    {"$match": {
        "reportedDateTime": {
            "$gte": datetime(2010, 1, 1),
            "$lt": datetime(2015, 1, 1)
        }
    }},
    # Přidání pole "den" (datum bez času) a "rok"
    {"$addFields": {
        "den": {
            "$dateToString": {"format": "%Y-%m-%d", "date": "$reportedDateTime"}
        },
        "rok": {"$year": "$reportedDateTime"}
    }},
    # Seskupení podle dne a roku, spočítání počtu výjezdů v daný den
    {"$group": {
        "_id": {"rok": "$rok", "den": "$den"},
        "pocet_vyjezdu": {"$sum": 1}
    }},
    # Seskupení podle roku, nalezení dne s největším počtem výjezdů
    {"$sort": {
        "_id.rok": 1,
        "pocet_vyjezdu": -1
    }},
    {"$group": {
        "_id": "$_id.rok",
        "den": {"$first": "$_id.den"},
        "pocet_vyjezdu": {"$first": "$pocet_vyjezdu"}
    }},
    # Výstupní formát
    {"$project": {
        "_id": 0,
        "rok": "$_id",
        "den": 1,
        "pocet_vyjezdu": 1
    }},
    {"$sort": {"rok": 1}}
]

result = list(db.departures.aggregate(pipeline))

if result:
    for res in result:
        print(dumps(res, indent=2, ensure_ascii=False))
else:
    print("Žádné záznamy nenalezeny")

{
  "den": "2010-06-02",
  "pocet_vyjezdu": 156,
  "rok": 2010
}
{
  "den": "2011-09-05",
  "pocet_vyjezdu": 39,
  "rok": 2011
}
{
  "den": "2012-04-29",
  "pocet_vyjezdu": 61,
  "rok": 2012
}
{
  "den": "2013-06-25",
  "pocet_vyjezdu": 105,
  "rok": 2013
}
{
  "den": "2014-09-14",
  "pocet_vyjezdu": 86,
  "rok": 2014
}


In [118]:
from bson.json_util import dumps
from db_connection import get_db_connection
db = get_db_connection()

### Dny v týdnu s největším počtem výjezdů a typem onoho výjezdu (2025)
pipeline = [
    # Filtrování na roky 2010-2015
    {"$match": {
        "reportedDateTime": {
            "$gte": datetime(2025, 1, 1),
            "$lt": datetime(2026, 1, 1)
        }
    }},
    # Přidáme den v týdnu (1=neděle, ..., 7=sobota)
    {"$addFields": {
        "dayOfWeek": {"$dayOfWeek": "$reportedDateTime"}
    }},
    # Seskupíme podle dne v týdnu a typu výjezdu, spočítáme počet výjezdů
    {"$group": {
        "_id": {"dayOfWeek": "$dayOfWeek", "type": "$type"},
        "count": {"$sum": 1}
    }},
    # Seřadíme podle dne a počtu sestupně
    {"$sort": {
        "_id.dayOfWeek": 1,
        "count": -1
    }},
    # Pro každý den v týdnu vybereme typ s největším počtem výjezdů
    {"$group": {
        "_id": "$_id.dayOfWeek",
        "mostFrequentType": {"$first": "$_id.type"},
        "count": {"$first": "$count"}
    }},
    # Přehlednější projekce výsledku
    {"$project": {
        "_id": 0,
        "dayOfWeek": "$_id",
        "mostFrequentType": 1,
        "count": 1
    }},
    {"$sort": {"dayOfWeek": 1}}
]

result = list(db.departures.aggregate(pipeline))

if result:
    for res in result:
        print(dumps(res, indent=2, ensure_ascii=False))
else:
    print("Žádné záznamy nenalezeny")

{
  "mostFrequentType": "Požár",
  "count": 80,
  "dayOfWeek": 1
}
{
  "mostFrequentType": "Dopravní nehoda",
  "count": 101,
  "dayOfWeek": 2
}
{
  "mostFrequentType": "Dopravní nehoda",
  "count": 110,
  "dayOfWeek": 3
}
{
  "mostFrequentType": "Dopravní nehoda",
  "count": 135,
  "dayOfWeek": 4
}
{
  "mostFrequentType": "Dopravní nehoda",
  "count": 108,
  "dayOfWeek": 5
}
{
  "mostFrequentType": "Dopravní nehoda",
  "count": 133,
  "dayOfWeek": 6
}
{
  "mostFrequentType": "Požár",
  "count": 100,
  "dayOfWeek": 7
}


In [120]:
from bson.json_util import dumps
from db_connection import get_db_connection
db = get_db_connection()

### Dny v týdnu s největším počtem výjezdů a typem onoho výjezdu (2025)
pipeline = [
    # Spojení s kolekcí sent_units
    {"$lookup": {
        "from": "sent_units",
        "localField": "departure_id",
        "foreignField": "departure_id",
        "as": "units"
    }},
    # Přidání počtu jednotek
    {"$addFields": {
        "unitsCount": {"$size": "$units"}
    }},
    # Seřazení jednotek podle callDateTime vzestupně
    {"$addFields": {
        "units": {"$sortArray": {"input": "$units", "sortBy": {"callDateTime": 1}}}
    }},
    # Seřazení výjezdů podle počtu jednotek sestupně
    {"$sort": {"unitsCount": -1}},
    # Výběr pouze jednoho výjezdu s nejvíce jednotkami
    {"$limit": 1},
    # Výpis všech dostupných informací o výjezdu a jednotkách
    {"$project": {
        "_id": 0,
        "departure_id": 1,
        "reportedDateTime": 1,
        "state": 1,
        "type": 1,
        "subType": 1,
        "description": 1,
        "region_url": 1,
        "unitsCount": 1,
        "units": 1
    }}
]

result = list(db.departures.aggregate(pipeline))

if result:
    for res in result:
        print(dumps(res, indent=2, ensure_ascii=False))
else:
    print("Žádné záznamy nenalezeny")

{
  "departure_id": 363700062,
  "reportedDateTime": {
    "$date": "2021-10-05T19:11:16Z"
  },
  "state": "Uzavřená OS",
  "type": "Požár",
  "subType": "Lesní porost",
  "description": "Zasahovali jsme u požáru stromů a trávy v lese. Nasadili jsme několik vodních proudů. Požářiště rozděleno na dva úseky. 1:27 lokalizace. Škoda způsobená požárem je 400.000 Kč, uchráněné hodnoty 10 mil. Kč. Likvidace 18:26.",
  "region_url": "https://udalosti.firebrno.cz",
  "units": [
    {
      "_id": {
        "$oid": "6829cb045795476d4c1b4497"
      },
      "departure_id": 363700062,
      "type": "cisternová automobilová stříkačka < 30",
      "unit": "Vracov",
      "count": 2,
      "currentCount": 0,
      "callDateTime": {
        "$date": "2021-10-05T21:12:00Z"
      }
    },
    {
      "_id": {
        "$oid": "6829cb045795476d4c1b4498"
      },
      "departure_id": 363700062,
      "type": "přenosná elektrocentrála s výkonem nad 5 kW",
      "unit": "Vracov",
      "count": 1,
      "cu

In [122]:
from bson.json_util import dumps
from db_connection import get_db_connection
db = get_db_connection()

### Konfigurace - Výpis všech typů výjezdů
result = db.departures.distinct("type", {"type": {"$ne": None}})

if result:
    for res in result:
        print(dumps(res, indent=2, ensure_ascii=False))
else:
    print("Žádné záznamy nenalezeny")

"Dopravní nehoda"
"Formálně založená událost"
"Jiná událost"
"Požár"
"Technická pomoc"
"Technologický test"
"Záchrana osob a zvířat"
"Únik nebezpečných látek"


In [124]:
from bson.json_util import dumps
from db_connection import get_db_connection
db = get_db_connection()

### Konfigurace - Výpis všech okresů v Jihomoravském kraji, o kterých je minimálně jeden záznam v kolekci addresses
result = db.addresses.distinct("district.name", {"region.name": "Jihomoravský"})

if result:
    for res in result:
        print(dumps(res, indent=2, ensure_ascii=False))
else:
    print("Žádné záznamy nenalezeny")

"Blansko"
"Brno-město"
"Brno-venkov"
"Břeclav"
"Hodonín"
"Vyškov"
"Znojmo"


In [152]:
from bson.json_util import dumps
from db_connection import get_db_connection
db = get_db_connection()

### Konfigurace - Seznam všech jednotek, které jsou v kolekci sent_units
result = list(sorted(db.sent_units.distinct("unit")))

if result:
    print(len(result))
    # for res in result:
        # print(dumps(res, indent=2, ensure_ascii=False))
else:
    print("Žádné záznamy nenalezeny")

535


In [134]:
from bson.json_util import dumps
from db_connection import get_db_connection
db = get_db_connection()

### Vyhledání výjezdů v konkrétním okrese
result = db.addresses.find({ "district.name": "Pelhřimov" })

if result:
    for res in result:
        print(dumps(res, indent=2, ensure_ascii=False))
else:
    print("Žádné záznamy nenalezeny")

{
  "_id": {
    "$oid": "6829cb01f2fef90ffc37680e"
  },
  "departure_id": 118053062,
  "region": {
    "id": 108,
    "name": "Vysočina"
  },
  "district": {
    "id": 3304,
    "name": "Pelhřimov"
  },
  "municipality": "Komorovice",
  "municipalityPart": null,
  "municipalityWithExtendedCompetence": "Humpolec",
  "street": null,
  "gis1": "685222",
  "gis2": "1115496",
  "preplanned": false,
  "road": null
}


In [151]:
from bson.json_util import dumps
from db_connection import get_db_connection
db = get_db_connection()

### Vyhledání výjezdů s chybějícími souřadnicemi
result = list(db.addresses.find({"$or": [ 
    { "gis1": None }, 
    { "gis2": None } 
]}))

if result:
    print(len(result))
    # for res in result:
        # print(dumps(res, indent=2, ensure_ascii=False))
else:
    print("Žádné záznamy nenalezeny")

1748


In [3]:
from bson.json_util import dumps
from db_connection import get_db_connection
db = get_db_connection()

### Vytvoření indexů

db.departures.create_index({"departure_id": 1}, unique=True);
db.addresses.create_index({"departure_id": 1}, unique=True);
db.sent_units.create_index({"departure_id": 1});
db.merged_address_units.create_index("departure_id", unique=True);

In [2]:
import os
from db_connection import get_db_client

### Konfigurace - zapnutí shardingu nad databází

mongo_database = os.getenv('DATABASE_NAME')
client = get_db_client()

client.admin.command("enableSharding", mongo_database)


{'ok': 1.0,
 '$clusterTime': {'clusterTime': Timestamp(1747643966, 1),
  'signature': {'hash': b'\x13\x16@\xf1@\x1a\x17\x07\xe8v\x9c\xe9\x15\x88\x02\x0fB\x87~\x1e',
   'keyId': 7505753227216814103}},
 'operationTime': Timestamp(1747643966, 1)}

In [6]:
import os
from db_connection import get_db_client

### Konfigurace - zapnutí shardingu nad kolekcí s klíčem departure_id - hashed - rozprostře data rovnoměrně po všech shardech

mongo_database = os.getenv('DATABASE_NAME')
client = get_db_client()

client.admin.command("shardCollection", f'{mongo_database}.departures', key={"departure_id": "hashed"})
client.admin.command("shardCollection", f'{mongo_database}.addresses', key={"departure_id": "hashed"})
client.admin.command("shardCollection", f'{mongo_database}.sent_units', key={"departure_id": "hashed"})


{'collectionsharded': 'fire_departures_db.sent_units',
 'ok': 1.0,
 '$clusterTime': {'clusterTime': Timestamp(1747646097, 2),
  'signature': {'hash': b'\x82\x13$\xca?\xd4)d\x8f\xb2h\x84\x94\xd1wV?\xb6l7',
   'keyId': 7505753227216814103}},
 'operationTime': Timestamp(1747646097, 2)}

In [13]:
from bson.json_util import dumps
from db_connection import get_db_connection
db = get_db_connection()

### Vytvoření kolekce s validací

validator = {
    "$jsonSchema": {
        "bsonType": "object",
        "required": ["departure_id", "reportedDateTime", "state"],
        "properties": {
            "departure_id": {"bsonType": "int"},
            "reportedDateTime": {"bsonType": "date"},
            "state": {"bsonType": "string"},
            "type": {"bsonType": ["string", "null"]},
            "subType": {"bsonType": ["string", "null"]},
            "description": {"bsonType": ["string", "null"]},
            "region_url": {"bsonType": ["string", "null"]}
        }
    }
}

db.create_collection(
    "departures1",
    validator=validator,
    validationLevel="strict",
    validationAction="error"
)

### Smazání kolekce

db.departures1.drop()